In [ ]:
#Importin neccessry libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
import nltk
import re
import numpy as np
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
ps = PorterStemmer()
import pickle

In [57]:
data = pd.read_csv('.\dataset\dataset.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [59]:
data.dropna(subset = ["text"],inplace = True)
data.reset_index(inplace = True,drop = True)

In [72]:
data[139:145]

,id,title,author,text,label
139,139,It’s Official: Simone Biles Is the World’s Bes...,"Victor Mather, Doug Mills and Chang W. Lee","RIO DE JANEIRO — Simone Biles, already cons...",0
140,140,NaN,Anonymous,There is a lot more than meets the eye to this...,1
141,141,It Literally Hurts My Brain to Read the Econom...,beforeitsnews.com,(Before It's News)\n(Don Boudreaux)\nTweet\nHe...,1
142,143,U.N. Secretary General Complains That The ‘Mas...,Geoffrey Grider,U.N. Secretary General Complains That The ‘M...,1
143,144,Trump Bollywood Ad Meant To Sway Indian Americ...,T Steelman,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
144,145,FBI Finds Previously Unseen Hillary Clinton Em...,Zero Hedge,\nCrushing the hope-filled “it’s just a backup...,1


In [ ]:
#Declaring requried variables

ps = PorterStemmer()
vocab_size = 238052
max_length = 500
embedding_dim= 100

In [100]:
#Preprocessing
corpus = []
for i in range(0, len(data)):
    #regex to remove emojis and special characters
    review = re.sub('[^a-zA-Z]', ' ', data['text'][i])
    #Converts text to lower cae
    review = review.lower()
    review = review.split()
    #Performs stemming on the corpus and removes stopwords
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [109]:
#One hot encoding of the processed text
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 

In [121]:
#Padding the text to make the input of same size and shape
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=max_length)

In [123]:
#Splitting data into training and testing sets
X_final = np.array(embedded_docs)
y_final = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=609)

In [124]:
#Stacked LSTM model

model = Sequential()

model.add(Embedding(vocab_size,embedding_dim,input_length=max_length))

model.add(LSTM(units=32,  dropout=0.2, recurrent_dropout=0.2,return_sequences = True))

model.add(LSTM(units=32,  dropout=0.2, recurrent_dropout=0.2,return_sequences = False))


model.add(Dense(64))
model.add(Activation("relu"))

# output layer using sigmoid as activation method (0-1)

model.add(Dense(1)) # classification problem: so output = 1)
model.add(Activation("sigmoid"))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Summary of the built model...
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 500, 100)          23805200  
                                                                 
 lstm_18 (LSTM)              (None, 500, 32)           17024     
                                                                 
 lstm_19 (LSTM)              (None, 32)                8320      
                                                                 
 dense_15 (Dense)            (None, 64)                2112      
                                                                 
 activation_14 (Activation)  (None, 64)                0         
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                                 
 activation_15 (Activat

In [125]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64,verbose = 1)

Epoch 1/10
218/218 [==============================] - 217s 965ms/step - loss: 0.3098 - accuracy: 0.8719 - val_loss: 0.1874 - val_accuracy: 0.9278
Epoch 2/10
218/218 [==============================] - 209s 957ms/step - loss: 0.0877 - accuracy: 0.9703 - val_loss: 0.1744 - val_accuracy: 0.9397
Epoch 3/10
218/218 [==============================] - 204s 937ms/step - loss: 0.0303 - accuracy: 0.9888 - val_loss: 0.2697 - val_accuracy: 0.9441
Epoch 4/10
218/218 [==============================] - 204s 936ms/step - loss: 0.0169 - accuracy: 0.9943 - val_loss: 0.2844 - val_accuracy: 0.9335
Epoch 5/10
218/218 [==============================] - 205s 941ms/step - loss: 0.0221 - accuracy: 0.9923 - val_loss: 0.2747 - val_accuracy: 0.9409
Epoch 6/10
218/218 [==============================] - 207s 948ms/step - loss: 0.0098 - accuracy: 0.9974 - val_loss: 0.3133 - val_accuracy: 0.9256
Epoch 7/10
218/218 [==============================] - 206s 945ms/step - loss: 0.0051 - accuracy: 0.9985 - val_loss: 0.3327 -

In [126]:
model.save("Final_model.h5")

In [6]:
# should be fake (1)
sample = "BenoÃ®t Hamon Wins French Socialist Partyâ€™s Presidential Nomination - The New York Times"

In [ ]:
# should be true (0)
sample = "Russian Researchers Discover Secret Nazi Military Base â€˜Treasure Hunterâ€™ in the Arctic "

In [3]:
model = tf.keras.models.load_model("F:/Durham_College_AI/2- semester/AI in enterprise/Final_project/Final_model.h5")

In [10]:
def preprocess(tweet):
    test = []
    sample = [tweet]
    for i in range(0, len(sample)):
        review = re.sub('[^a-zA-Z]', ' ', sample[i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        test.append(review)
        one_hot_enc =[one_hot(words,vocab_size)for words in test]
        pad_seq = pad_sequences(one_hot_enc,padding='pre',maxlen=max_length)
    return pad_seq

In [11]:
y = preprocess(sample)

In [12]:
if model.predict(x=y) > 0.5:
    print("Fake")
else:
    print("True")

Fake
